In [1]:
import pandas as pd

In [2]:
def make_sales():
    '''function to make df for sales data'''
    sales = pd.read_csv("https://raw.githubusercontent.com/VallieTracy/opioids_project/master/Resources/opioidPrescriptionSales.csv")
    sales['Type'] = sales['Oxycodone / Hydrocodone'].str.replace('/', '')
    #sales['Data Type'] = sales['Data Type'].str.replace('  ', '_')
    sales = sales[["Fips", "Location","TimeFrame","Type", "Data", ]]
    sales= sales.query("Type == 'Oxycodone'")
    #sales = sales.groupby("Data").median().reset_index()
    #sales.query("TimeFrame == 2018")
    #return sales.query("TimeFrame == 2018 and Type == 'Oxycodone'")
    sales = sales.rename(columns={"Data":"SalesData"})
    return sales
make_sales()




,Fips,Location,TimeFrame,Type,SalesData
0,1,Alabama,2000,Oxycodone,6.74399
1,1,Alabama,2001,Oxycodone,7.79399
2,1,Alabama,2002,Oxycodone,7.80123
3,1,Alabama,2003,Oxycodone,9.26128
4,1,Alabama,2004,Oxycodone,9.10411
...,...,...,...,...,...
1914,56,Wyoming,2014,Oxycodone,20.63788
1915,56,Wyoming,2015,Oxycodone,21.83413
1916,56,Wyoming,2016,Oxycodone,20.73213
1934,56,Wyoming,2017,Oxycodone,18.12706


In [3]:


def make_deaths():
    '''function to make df for deaths data'''
    deaths = pd.read_csv("https://raw.githubusercontent.com/VallieTracy/opioids_project/master/Resources/drugPoisoningDeaths.csv")
    #sales['Type'] = sales['Oxycodone / Hydrocodone'].str.replace('/', '')
    #sales['Data Type'] = sales['Data Type'].str.replace('  ', '_')
    deaths = deaths[["Fips", "Location","TimeFrame","Drug Type", "Data", ]]
    #deaths= deaths.query("TimeFrame == 2018") #and 'Drug Type' == 'All opioids')
    
    deaths= deaths.query("`Drug Type` == 'All opioids'")
    #deaths = deaths.groupby("Data").median().reset_index()
    #grouped = deaths.groupby(pd.qcut(deaths['Data'], q=10))#.reset_index()
    #sales.query("TimeFrame == 2018")
    #return sales.query("TimeFrame == 2018 and Type == 'Oxycodone'")
    deaths = deaths.rename(columns={"Data":"DeathsData"})
    deaths = deaths.rename(columns={"Drug Type":"ODType"})
    #grouped = grouped.rename(columns={"Data":"Deaths Data"})
    return deaths
make_deaths()



,Fips,Location,TimeFrame,ODType,DeathsData
91,1,Alabama,2000,All opioids,0.98993
93,1,Alabama,2001,All opioids,1.29709
95,1,Alabama,2002,All opioids,1.61198
97,1,Alabama,2003,All opioids,1.10769
99,1,Alabama,2004,All opioids,1.84700
...,...,...,...,...,...
6769,56,Wyoming,2014,All opioids,9.29636
6771,56,Wyoming,2015,All opioids,7.89044
6773,56,Wyoming,2016,All opioids,8.74569
6775,56,Wyoming,2017,All opioids,8.73564


In [62]:
def make_merge():
    '''function to combine death and sales data'''
    sales = make_sales()
    deaths = make_deaths()
    #merged = deaths.merge(sales, on="Fips")
    merged= pd.merge(deaths, sales, left_on=['Fips','Location', 'TimeFrame'], right_on=['Fips','Location', 'TimeFrame'])
    merged = merged[['Location','Fips','TimeFrame','SalesData','DeathsData']]
    merged = merged.dropna()
    # remove 5% of samples
    df1 = merged.query('DeathsData < 16 & SalesData < 29')
    df_test = df1[["Location","TimeFrame","SalesData","DeathsData"]]




    #eturn merged
    return df_test
make_merge()
# WRITE FUNCTION TO NEW DF
df= make_merge()
df

,Location,TimeFrame,SalesData,DeathsData
0,Alabama,2000,6.74399,0.98993
1,Alabama,2001,7.79399,1.29709
2,Alabama,2002,7.80123,1.61198
3,Alabama,2003,9.26128,1.10769
4,Alabama,2004,9.10411,1.84700
...,...,...,...,...
964,Wyoming,2014,20.63788,9.29636
965,Wyoming,2015,21.83413,7.89044
966,Wyoming,2016,20.73213,8.74569
967,Wyoming,2017,18.12706,8.73564


In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming your data is stored in a pandas DataFrame called 'df'
#df_encoded = pd.get_dummies(df, columns=['Location'])

# Separate the features and labels
X = df[['Fips','TimeFrame','SalesData']] #+ [col for col in df_encoded.columns if col.startswith('Location_')]]
y = df[['DeathsData']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [63]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Assuming your data is stored in a pandas DataFrame called 'df'
X = df[['Fips','TimeFrame','SalesData']]  # Features (Opioid sales)
y = df['DeathsData']  # Target variable (Opioid deaths)

# Initialize the Linear Regression model
model = LinearRegression()

# Train the model on the entire dataset
model.fit(X, y)

# Make predictions using the entire dataset
predictions = model.predict(X)

# Add the predicted values to the original DataFrame
df['PredictedDeaths'] = predictions
predictions
df['diff'] = df['DeathsData'] - df['PredictedDeaths']
df.query('Location == "Ohio"')

KeyError: "['Fips'] not in index"

In [40]:
from sklearn.linear_model import LinearRegression

# Assuming you have already split and preprocessed the data

# Initialize the Linear Regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
predictions = model.predict(X_test)
predictions


array([[ 6.33343386],
       [-0.29291075],
       [14.77529511],
       [ 4.99229254],
       [ 9.00650167],
       [ 6.29579411],
       [12.88755073],
       [10.10893708],
       [12.29442763],
       [12.51759037],
       [ 5.93527362],
       [12.31713688],
       [ 7.58484586],
       [ 6.15454098],
       [10.66890234],
       [ 2.28867203],
       [ 9.24564732],
       [11.31037521],
       [11.93923534],
       [ 6.79178148],
       [ 8.6334121 ],
       [10.75170184],
       [12.69917847],
       [11.93542671],
       [ 8.51819686],
       [12.99953112],
       [14.90294146],
       [ 8.83879653],
       [ 3.90483349],
       [ 5.10222092],
       [12.56988325],
       [11.95451705],
       [10.04440831],
       [12.13418838],
       [ 5.81930057],
       [ 9.61750932],
       [10.13325315],
       [ 4.11187724],
       [11.03366223],
       [11.12064876],
       [ 9.53131329],
       [11.10337794],
       [10.99134079],
       [13.81703324],
       [ 9.17043219],
       [ 2

In [47]:
predictions
#df_encoded

 #Add the predicted values to the original DataFrame
#df_encoded['PredictedDeaths'] = predictions


# Add the predicted values to the original DataFrame
X_test['PredictedDeaths'] = predictions
#X_test
X_train



,Fips,TimeFrame,SalesData
60,5,2003,7.46912
307,20,2003,7.81420
511,30,2017,15.39811
866,50,2011,19.42688
402,25,2003,14.20720
...,...,...,...
112,8,2017,17.74817
277,18,2011,16.07516
889,51,2015,18.42387
443,27,2006,8.76522


In [17]:
X_test

,TimeFrame,SalesData,Location_Alabama,Location_Alaska,Location_Arizona,Location_Arkansas,Location_California,Location_Colorado,Location_Connecticut,Location_Delaware,...,Location_South Dakota,Location_Tennessee,Location_Texas,Location_Utah,Location_Vermont,Location_Virginia,Location_Washington,Location_West Virginia,Location_Wisconsin,Location_Wyoming
859,2004,12.03303,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
76,2000,2.57246,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
643,2016,26.54400,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
518,2005,7.80776,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
729,2007,20.72922,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,2002,7.80123,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
865,2010,19.85312,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
762,2002,9.86458,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
556,2005,15.52598,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
